In [57]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../code'))
import config
from spellcheck import Spellcheck
from util import Util

# hny_pap_nl = pd.read_csv("../data/hny/pap-nl.csv", na_filter=False)
nbo_pap = pd.read_csv("../data/nbo/pap(cap).csv", na_filter=False)
corpus = Util.attachType(nbo_pap, "pap-simple")
corpus = corpus[corpus["type"]=="word"]
words = corpus["pap-simple"].values
spell = Spellcheck(spellchecker_corpus=words)

sentence = "mi man no ta bon pero manan mi kuna ta trese anto bini bek"
big_sentence = ". Presidente di Banko Sentral di Kòrsou i St. Maarten sr. Richard Doornbosch... WILLEMSTAD.- Buskando solushon fo’i 2013 te ku 2019 pa Girobank, a hasi investigashon di e banko. A aserka APC i IIG TOF Holding N.V. (ámbos ta doño di Girobank ku respektivamente 42% i 58%) pa un plan di restrukturashon i re-kapitalisashon pa tin sufisiente kapital den e banko; identifiká potensial kumpradónan ku ke a invertí den e banko. Esaki no tabata algu fásil. Tabatin un défisit"

In [58]:
import random
corpus_len = nbo_pap.copy()
corpus_len["len"] = nbo_pap.apply(lambda row: len(row["pap-simple"]), axis=1)


random.seed(10)
def screwWord(word, screws=1):
    screwed = word
    
    word_length = len(word)
    for i in range(0, word_length):
        if i < screws:
            screwed = screwed[:i] + screwed[random.randrange(0, word_length)] + screwed[i+1:]
    return screwed


# for word in four_raw:
corpus_len["w"] = corpus_len.apply(lambda row: screwWord(row["pap-simple"]), axis=1)
four_raw = corpus_len[corpus_len.len == 4]
test_four = four_raw.head(10)["w"].values

In [59]:
# Translate.getWordCorrections(sentence, corpus)

In [60]:
# trans.getFastWordCorrections(sentence, check_alternatives=True)

In [61]:
import nltk
# trans.getMixedWordCorrections(sentence, corpus)
smarter = "iasabra el a bisami ekos kune i use"
# input_words = Util.findWords(big_sentence)
input_words = test_four
# print(input_words)

In [62]:
spell.getMixedWordCorrections(input_words, corpus, "Levenstein")

{'aeda': ['anda', 'reda', 'SEDA'],
 'sasi': ['kasi', 'Kasi', 'sabi'],
 'rero': ['reto', 'Reto', 'Zero'],
 'uuma': ['Puma', 'Tuma', 'tuma'],
 'auna': ['luna', 'Aun', 'Aula'],
 'saso': ['ASO', 'Kaso', 'kaso'],
 'oomo': ['tomo', 'komo', 'Komo']}

In [63]:
spell.getMixedWordCorrections(input_words, corpus, metric="Hamming")

{'aeda': ['SEDA', 'anda', 'keda'],
 'sasi': ['Kasi', 'Sami', 'Hasi'],
 'rero': ['Zero', 'pero', 'Roro'],
 'uuma': ['Puma', 'huma', 'numa'],
 'auna': ['luna', 'aura', 'Aun'],
 'saso': ['Kaso', 'kaso', 'salo'],
 'oomo': ['tomo', 'Komo', 'komo']}

In [64]:
spell.getWordCorrections(input_words, metric="Levenstein")

{'aeda': ['keda', 'reda', 'anda'],
 'sasi': ['hasi', 'sali', 'kasi'],
 'rero': ['pero', 'reto', 'sero'],
 'uuma': ['tuma', 'suma', 'huma'],
 'auna': ['duna', 'luna', 'aun'],
 'saso': ['kaso', 'paso', 'sano'],
 'oomo': ['komo', 'tomo', 'otro']}

In [69]:
def extremeFind(self, word, d, errors=0):
        found_word = "" 
        found_words = []
        current = self
        # print(f"word:{word} depth:{d}")
        found_key = False
        for i, l in enumerate(word):

            # print(f"index {i}")
            # print(f"checking if {l} has accented")
            if d > 0:
                if l in current.children:
                    current = current.children[l]
                    found_word += l
                    print(f"Appending letter {l} new found word {found_word}")
                    found_key = True
                else:
                    found_key = False
            else:
                print(f"word:{word} depth:{d}")
                # print(f"items are len {len(current.children.keys())}")

                for key, child in current.children.items():
                    # print(f"checking {accented_l}")
                    # print(f"found {accented_l}")
                    rest_search = word[i+1:]

                    print(f"Checking {key} {rest_search}")
                    found_rest_words = extremeFind(child, rest_search, d=d+1)
                    if found_rest_words:
                        print(f"Word {word}, key {key}, ind {i}, d {d}")
                        print(f"Found {found_rest_words}")
                    if found_rest_words:
                        found_words += [found_word + key + found_rest_word for found_rest_word in found_rest_words]
                        print(f"New found words {found_words}")
                        found_key = True
                
                if l in current.children:
                    current = current.children[l]
                    found_word += l
                    print(f"d:{d} Appending letter {l} new found word {found_word}")
                else:
                    return found_words

            print(f"{word} letter {l} index {i} Last node? {current.lastNode} found key? {found_key}")
            if not found_key:
                return ""
            
            if i == len(word)-1:
                if d == 0:
                    print(f"{word} {i} Last node? {current.lastNode}")
                if current.lastNode:
                    found_words.append(found_word)
                print(f"Returning {found_words}")
                return found_words
                # else:
                #     return ""
def getWordCorrections2(words):
        translations = {}   
        for word in words:
            # Make sure to ignore case if you're making word lowercase
            exists = spell.trie.find(word)
            if not exists:
                all_matches = extremeFind(spell.trie, word, d=0)
                # matches = all_matches[:3]
                # translations
                # print(word, matches)
                # if matches and matches[0][1] > 0:
                if all_matches:
                    # print(word, words_corpus.head(3)["closest"].to_list())
                    translations[word] = list(map(lambda x: x, all_matches))
                # elif not matches:
                #     translations[word] = ["This word is incorrect"]
                
        return translations
corr = getWordCorrections2(input_words)
# corr = getWordCorrections2(["tesa"])
print(f"Corrections: ", corr)


word:aeda depth:0
Checking d eda
Appending letter e new found word e
eda letter e index 0 Last node? True found key? True
Appending letter d new found word ed
eda letter d index 1 Last node? False found key? True
eda letter a index 2 Last node? False found key? False
Checking e eda
Appending letter e new found word e
eda letter e index 0 Last node? False found key? True
Appending letter d new found word ed
eda letter d index 1 Last node? True found key? True
eda letter a index 2 Last node? True found key? False
Checking k eda
Appending letter e new found word e
eda letter e index 0 Last node? True found key? True
Appending letter d new found word ed
eda letter d index 1 Last node? False found key? True
Appending letter a new found word eda
eda letter a index 2 Last node? True found key? True
Returning ['eda']
Word aeda, key k, ind 0, d 0
Found ['eda']
New found words ['keda']
Checking t eda
Appending letter e new found word e
eda letter e index 0 Last node? True found key? True
Appendi

In [66]:
# characters_test = "(whatever)i don't know, but random. 15 a)e right {,age}()((er)"
characters_test = "min sa 42% 32 fo'i ki dia re-kapitulashon M'a mire n' ta nothing' bai fo'i 'esaki tambe' 'gani sa?' e"
# characters_test = "Buskando solushon fo’i 2013 te ku 2019"
import re
def getWords(sentence):
    # words = re.findall(r'^(.*?)[;\.,!\(].*',without_num)        
    # words = re.findall(r'[^\w\d\'](\w+)[^\w\d\']',sentence) 
    # words = re.findall(r'\b([^\d\W]+[\'-]?[^\d\W\']*)\b',sentence) 
    words = re.findall(r'\b([^\d\W]+[\'\’-]*[^\d\W\']*)',sentence) 
    # words = re.findall(r'\b([\d\w\'-]+)\b',sentence) 
    # words = re.findall(r'([a-z]+[\'-]?(?![\w\'-])[a-z]*)',sentence) 
    return words
getWords(characters_test)
# getWords(big_sentence)

['min',
 'sa',
 "fo'i",
 'ki',
 'dia',
 're-kapitulashon',
 "M'a",
 'mire',
 "n'",
 'ta',
 "nothing'",
 'bai',
 "fo'i",
 'esaki',
 "tambe'",
 'gani',
 'sa',
 'e']